In [137]:
# This serves as a template which will guide you through the implementation of this task.  It is advised
# to first read the whole template and get a sense of the overall structure of the code before trying to fill in any of the TODO gaps
# First, we import necessary libraries:
from pprint import pprint

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.preprocessing import scale
from tqdm import tqdm
from torch.utils.data import TensorDataset, DataLoader
from sklearn.linear_model import LinearRegression, Lasso

In [76]:
seed = 0
np.random.seed(seed)
torch.manual_seed(seed)

In [77]:
def load_data():
    """
    This function loads the data from the csv files and returns it as numpy arrays.

    input: None

    output: x_pretrain: np.ndarray, the features of the pretraining set
            y_pretrain: np.ndarray, the labels of the pretraining set
            x_train: np.ndarray, the features of the training set
            y_train: np.ndarray, the labels of the training set
            x_test: np.ndarray, the features of the test set
    """
    x_pretrain = pd.read_csv("public/pretrain_features.csv.zip", index_col="Id", compression='zip').drop("smiles",
                                                                                                         axis=1).to_numpy()
    y_pretrain = pd.read_csv("public/pretrain_labels.csv.zip", index_col="Id", compression='zip').to_numpy().squeeze(-1)
    x_train = pd.read_csv("public/train_features.csv.zip", index_col="Id", compression='zip').drop("smiles",
                                                                                                   axis=1).to_numpy()
    y_train = pd.read_csv("public/train_labels.csv.zip", index_col="Id", compression='zip').to_numpy().squeeze(-1)
    x_test = pd.read_csv("public/test_features.csv.zip", index_col="Id", compression='zip').drop("smiles", axis=1)
    return x_pretrain, y_pretrain, x_train, y_train, x_test

In [78]:
x_pretrain, y_pretrain, x_train, y_train, x_test = load_data()
print("Data loaded!")

Data loaded!


In [194]:
class Net(nn.Module):
    """
    The model class, which defines our feature extractor used in pretraining.
    """

    def __init__(self):
        """
        The constructor of the model.
        """
        super().__init__()
        # TODO: Define the architecture of the model. It should be able to be trained on pretraining data
        # and then used to extract features from the training and test data.

        # activation = nn.ReLU()

        # THINGS TO INCLUDE: BATCH NORM, DROPOUT
        self.encoder = nn.Sequential(
            nn.Linear(1000, 500), nn.BatchNorm1d(500), nn.ReLU(),
            nn.Linear(500, 250), nn.BatchNorm1d(250), nn.ReLU(),
            nn.Linear(250, 100), nn.BatchNorm1d(100),
            # nn.ReLU(),
            # nn.Linear(100, 50), nn.ReLU(),
            # nn.Linear(50, 10), nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            # nn.Linear(10, 50), nn.ReLU(),
            # nn.Linear(50, 100), nn.ReLU(),
            nn.Linear(100, 250), nn.BatchNorm1d(250), nn.ReLU(),
            nn.Linear(250, 500), nn.BatchNorm1d(500), nn.ReLU(),
            nn.Linear(500, 1000)
        )
        self.last_linear = nn.Sequential(nn.Linear(100, 1))

    def forward(self, x):
        """
        The forward pass of the model.
        input: x: torch.Tensor, the input to the model

        output: x: torch.Tensor, the output of the model
        """
        # TODO: Implement the forward pass of the model, in accordance with the architecture
        # defined in the constructor.
        x = self.encoder(x)
        y = self.last_linear(x)
        x = self.decoder(x)
        return x, y

In [195]:
def make_feature_extractor(x, y, batch_size=256, eval_size=1000):
    """
    This function trains the feature extractor on the pretraining data and returns a function which
    can be used to extract features from the training and test data.

    input: x: np.ndarray, the features of the pretraining set
              y: np.ndarray, the labels of the pretraining set
                batch_size: int, the batch size used for training
                eval_size: int, the size of the validation set

    output: make_features: function, a function which can be used to extract features from the training and test data
    """
    # Pretraining data loading
    in_features = x.shape[-1]
    x_tr, x_val, y_tr, y_val = train_test_split(x, y, test_size=eval_size, random_state=0, shuffle=True)
    x_tr, x_val = torch.tensor(x_tr, dtype=torch.float), torch.tensor(x_val, dtype=torch.float)
    y_tr, y_val = torch.tensor(y_tr, dtype=torch.float), torch.tensor(y_val, dtype=torch.float)

    # model declaration
    model = Net()
    model.train()

    # TODO: Implement the training loop. The model should be trained on the pretraining data. Use validation set
    # to monitor the loss.

    criterion_decoded = nn.MSELoss()
    criterion_predictions = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
    n_epochs = 5

    losses = []
    valid_losses = []

    train_loader = DataLoader(
        dataset=TensorDataset(x_tr, y_tr),
        batch_size=batch_size,
        shuffle=True
    )

    valid_loader = DataLoader(
        dataset=TensorDataset(x_val, y_val),
        batch_size=batch_size,
        shuffle=True
    )

    for epoch in range(n_epochs):
        train_loss_epoch = []
        valid_loss_epoch = []
        with tqdm(train_loader, unit="batch") as tepoch:
            for data, target in tepoch:
                tepoch.set_description(f"Epoch {epoch} train")

                optimizer.zero_grad()

                decoded_features, predictions = model(data)
                predictions = predictions.squeeze()

                loss_decoded = criterion_decoded(decoded_features, data)
                loss_predictions = criterion_predictions(predictions, target)

                a = 0.1
                train_loss = a*loss_predictions + (1-a)*loss_decoded

                train_loss_epoch.append(train_loss.item())

                train_loss.backward()
                optimizer.step()

                tepoch.set_postfix({'Train loss': train_loss.item()})

        train_loss_avg = np.mean(train_loss_epoch)

        with torch.no_grad():
            with tqdm(valid_loader, unit="batch") as tepoch:
                for valid_data, valid_target in tepoch:
                    tepoch.set_description(f"Epoch {epoch} valid")

                    valid_decoded_features, valid_predictions = model(valid_data)
                    valid_predictions = valid_predictions.squeeze()

                    valid_loss_decoded = criterion_decoded(valid_decoded_features, valid_data)
                    valid_loss_predictions = criterion_predictions(valid_predictions, valid_target)

                    valid_loss = a*valid_loss_predictions + (1-a)*valid_loss_decoded

                    valid_loss_epoch.append(valid_loss.item())

                    tepoch.set_postfix({'Validation loss': valid_loss.item()})

        valid_loss_avg = np.mean(valid_loss_epoch)


        losses.append(train_loss_avg)
        valid_losses.append(valid_loss_avg)

        print('Final train loss: ', train_loss_avg, 'Final valid loss: ', valid_loss_avg)

    def make_features(x):
        """
        This function extracts features from the training and test data, used in the actual pipeline
        after the pretraining.

        input: x: np.ndarray, the features of the training or test set

        output: features: np.ndarray, the features extracted from the training or test set, propagated
        further in the pipeline
        """
        model.eval()
        # TODO: Implement the feature extraction, a part of a pretrained model used later in the pipeline.
        model_no_last_layers = nn.Sequential(*list(model.children())[:-2])

        with torch.no_grad():
            if isinstance(x, pd.DataFrame):
                x = x.to_numpy()
            x = torch.tensor(x, dtype=torch.float)
            x_features = model_no_last_layers(x)

        return x_features

    return make_features

In [196]:
# Utilize pretraining data by creating feature extractor which extracts lumo energy
# features from available initial features
feature_extractor = make_feature_extractor(x_pretrain, y_pretrain)

x_train_transformed = feature_extractor(x_train).numpy()
x_test_transformed = feature_extractor(x_test).numpy()

y_pred = np.zeros(x_test.shape[0])

# STANDARDSCALER, FUNCTIONTRANSFORMER, etc.
x_train_transformed = scale(x_train_transformed)
x_test_transformed = scale(x_test_transformed)

Epoch 0 valid: 100%|██████████| 4/4 [00:00<00:00, 114.29batch/s, Validation loss=0.0274]


Final train loss:  0.21882997035087706 Final valid loss:  0.027624611277133226


Epoch 1 valid: 100%|██████████| 4/4 [00:00<00:00, 107.74batch/s, Validation loss=0.0226]


Final train loss:  0.024823771566540625 Final valid loss:  0.022794404067099094


Epoch 2 valid: 100%|██████████| 4/4 [00:00<00:00, 102.57batch/s, Validation loss=0.0193]


Final train loss:  0.020860203803749755 Final valid loss:  0.019452263135463


Epoch 3 valid: 100%|██████████| 4/4 [00:00<00:00, 117.64batch/s, Validation loss=0.0171]


Final train loss:  0.018026713747531176 Final valid loss:  0.017161923460662365


Epoch 4 valid: 100%|██████████| 4/4 [00:00<00:00, 111.09batch/s, Validation loss=0.0148]
C:\Users\Gustas\anaconda3\envs\iml-2023\lib\site-packages\sklearn\preprocessing\_data.py:240: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
C:\Users\Gustas\anaconda3\envs\iml-2023\lib\site-packages\sklearn\preprocessing\_data.py:259: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(
C:\Users\Gustas\anaconda3\envs\iml-2023\lib\site-packages\sklearn\preprocessing\_data.py:240: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
C:\Users\Gustas\anaconda3\envs\iml-2023\lib\site-packages\sklearn\prepr

Final train loss:  0.015698807740894456 Final valid loss:  0.0148190101608634


In [197]:
print('alpha =', 0)
pprint(cross_validate(LinearRegression(), x_train_transformed, y_train, cv=5, scoring='neg_root_mean_squared_error', return_train_score=True))
for alpha in [0.001, 0.01, 0.1, 1, 10, 100]:
    print('alpha =', alpha)
    pprint(cross_validate(Ridge(alpha=alpha), x_train_transformed, y_train, cv=5, scoring='neg_root_mean_squared_error', return_train_score=True))

alpha = 0
{'fit_time': array([0.04048324, 0.00200081, 0.0030005 , 0.00300002, 0.00199986]),
 'score_time': array([0.00200033, 0.        , 0.00099969, 0.        , 0.        ]),
 'test_score': array([-0.27480896, -0.3330765 , -0.40158268, -0.24590953, -0.3125876 ]),
 'train_score': array([-9.16513608e-06, -1.73120890e-06, -1.86957990e-05, -4.81487835e-06,
       -1.04403853e-05])}
alpha = 0.001
{'fit_time': array([0.01100492, 0.00101376, 0.00099945, 0.00098038, 0.00101662]),
 'score_time': array([0.00202036, 0.0010004 , 0.        , 0.        , 0.0010016 ]),
 'test_score': array([-0.24016498, -0.28581638, -0.37161898, -0.21469307, -0.29031317]),
 'train_score': array([-0.0101706 , -0.01134562, -0.0087398 , -0.01042438, -0.00882693])}
alpha = 0.01
{'fit_time': array([0.00098181, 0.0009973 , 0.00199962, 0.00100565, 0.00100136]),
 'score_time': array([0.0010221, 0.0010078, 0.       , 0.       , 0.       ]),
 'test_score': array([-0.20035152, -0.23958591, -0.25752826, -0.19343458, -0.25583726

C:\Users\Gustas\anaconda3\envs\iml-2023\lib\site-packages\sklearn\linear_model\_ridge.py:253: LinAlgWarning: Ill-conditioned matrix (rcond=4.15239e-08): result may not be accurate.
  dual_coef = linalg.solve(K, y, assume_a="pos", overwrite_a=False)
C:\Users\Gustas\anaconda3\envs\iml-2023\lib\site-packages\sklearn\linear_model\_ridge.py:253: LinAlgWarning: Ill-conditioned matrix (rcond=5.60894e-08): result may not be accurate.
  dual_coef = linalg.solve(K, y, assume_a="pos", overwrite_a=False)
C:\Users\Gustas\anaconda3\envs\iml-2023\lib\site-packages\sklearn\linear_model\_ridge.py:253: LinAlgWarning: Ill-conditioned matrix (rcond=4.01314e-08): result may not be accurate.
  dual_coef = linalg.solve(K, y, assume_a="pos", overwrite_a=False)
C:\Users\Gustas\anaconda3\envs\iml-2023\lib\site-packages\sklearn\linear_model\_ridge.py:253: LinAlgWarning: Ill-conditioned matrix (rcond=4.87656e-08): result may not be accurate.
  dual_coef = linalg.solve(K, y, assume_a="pos", overwrite_a=False)


In [204]:
pprint(cross_validate(Ridge(alpha=3), x_train_transformed, y_train, cv=5, scoring='neg_root_mean_squared_error', return_train_score=True))

{'fit_time': array([0.00200248, 0.00101161, 0.00101757, 0.0010035 , 0.00105572]),
 'score_time': array([0.00103378, 0.00099587, 0.00098181, 0.        , 0.        ]),
 'test_score': array([-0.20163826, -0.16007141, -0.21327105, -0.19572012, -0.23694184]),
 'train_score': array([-0.13733562, -0.14077238, -0.13100312, -0.13588726, -0.13342928])}


In [207]:
regression_model = Ridge(alpha=1)

In [208]:
regression_model.fit(x_train_transformed, y_train)
y_pred = regression_model.predict(x_test_transformed)

assert y_pred.shape == (x_test.shape[0],)
y_pred = pd.DataFrame({"y": y_pred}, index=x_test.index)
y_pred.to_csv("results.csv", index_label="Id")
print("Predictions saved, all done!")

Predictions saved, all done!
